In [3]:

%matplotlib inline
import os
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import SimpleRNN

from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import math
from sklearn.metrics import mean_squared_error
import FinanceDataReader as fdr
#특성 추가 ------------------------------------------
    #이동평균선
def get_MA(df):
    MA_26=df["Close"].rolling(26).mean()
    MA_52=df["Close"].rolling(52).mean()
    df=df.assign(MA_26=MA_26,MA_52=MA_52).dropna()
    
    return df

    
    #스토캐스틱
def get_stochastic(df, n=15, m=5, t=3):
    # n일중 최고가
    ndays_high = df.High.rolling(window=n, min_periods=1).max()
    # n일중 최저가
    ndays_low = df.Low.rolling(window=n, min_periods=1).min()
 
    # Fast%K 계산
    kdj_k = ((df.Close - ndays_low) / (ndays_high - ndays_low))*100
    # Fast%D (=Slow%K) 계산
    kdj_d = kdj_k.ewm(span=m).mean()
    # Slow%D 계산
    kdj_j = kdj_d.ewm(span=t).mean()
 
    # dataframe에 컬럼 추가
    df = df.assign(kdj_k=kdj_k, kdj_d=kdj_d, kdj_j=kdj_j).dropna()
    
    return df
   
    #시간
def get_time(df):
    time=np.linspace(0,10,len(df),endpoint=False).reshape(-1,1)
    df=df.assign(time=time)
    
    return df

In [5]:
KS11 = fdr.DataReader("KS11","2010-01-01","2019-01-01")
BATCH = 20

df = KS11
df=get_MA(df)
df=get_stochastic(df)
df=get_time(df)
df=df[0:-(len(df)%BATCH)]

MA_26=df["Close"].rolling(26).mean()
y_before=MA_26.dropna()
df=df[25:]

y=[]
for a in range(1,10001):
    try:
        y.append(np.where(df["Close"][-1+(BATCH*a)]>y_before[-1+(BATCH*a)],1,0)) # 19,39,59,...
        #y.append(np.where(df["Close"][-1+(BATCH*a)] > df["Close"][-1+(BATCH*(a-1))],1,0))
    except:
        break

df = df.values
df.astype('float32')
scaler = MinMaxScaler(feature_range=(0, 1))
df = scaler.fit_transform(df)

for x in range(100):
Xtrain_size=int(df.shape[0]/BATCH*0.7)*BATCH + (x*BATCH)

X_train=df[0:Xtrain_size]
X_test=df[Xtrain_size:Xtrain_size+BATCH]

y_train=y[ 0 : int(Xtrain_size/BATCH) ]
y_test=y[int(Xtrain_size/BATCH)]

# many to one 전처리 필요
y_train = np.reshape(y_train, (len(y_train), 1))
y_test = np.reshape(y_test, (1, 1))
X_train = np.reshape(X_train, (int(X_train.shape[0]/BATCH),BATCH, X_train.shape[1]))
X_test = np.reshape(X_test, (int(X_test.shape[0]/BATCH),BATCH, X_test.shape[1]))


X_train[:,BATCH-1,:]=0 #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
X_test[:,BATCH-1,:]=0 #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


# simple lstm network learning
model = Sequential()
model.add(LSTM(36, input_shape=(BATCH, 12)))
for i in range(5):
    model.add(Dense(36,activation='sigmoid'))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.fit(X_train, y_train, epochs=100, batch_size=16,validation_split=0.1,verbose=1)

model.predict(X_test)


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 66 samples, validate on 8 samples
Epoch 1/100
66/66 [==============================] - 0s 6ms/step - loss: 0.7694 - acc: 0.4394 - val_loss: 0.7147 - val_acc: 0.5000
Epoch 2/100
66/66 [==============================] - 0s 558us/step - loss: 0.7351 - acc: 0.4394 - val_loss: 0.7046 - val_acc: 0.5000
Epoch 3/100
66/66 [==============================] - 0s 574us/step - loss: 0.7190 - acc: 0.4394 - val_loss: 0.6989 - val_acc: 0.5000
Epoch 4/100
66/66 [==============================] - 0s 756us/step - loss: 0.7068 - acc: 0.4394 - val_loss: 0.6937 - val_acc: 0.5000
Epoch 5/100
66/66 [==============================] - 0s 816us/step - loss: 0.6969 - acc: 0.4394 - val_loss: 0.6931 - val_acc: 0.5000
Epoch 6/100
66/66 [==============================] - 0s 544us/step - loss: 0.6914 - acc: 0.5606 - val_loss: 0.6940 - v

66/66 [==============================] - 0s 816us/step - loss: 0.4792 - acc: 0.8333 - val_loss: 0.5736 - val_acc: 0.7500
Epoch 56/100
66/66 [==============================] - 0s 574us/step - loss: 0.5462 - acc: 0.7273 - val_loss: 0.5811 - val_acc: 0.8750
Epoch 57/100
66/66 [==============================] - 0s 529us/step - loss: 0.5538 - acc: 0.7121 - val_loss: 0.5625 - val_acc: 0.7500
Epoch 58/100
66/66 [==============================] - 0s 499us/step - loss: 0.4611 - acc: 0.8636 - val_loss: 0.5553 - val_acc: 0.7500
Epoch 59/100
66/66 [==============================] - 0s 786us/step - loss: 0.5004 - acc: 0.8182 - val_loss: 0.5493 - val_acc: 0.7500
Epoch 60/100
66/66 [==============================] - 0s 468us/step - loss: 0.4814 - acc: 0.8182 - val_loss: 0.5405 - val_acc: 0.7500
Epoch 61/100
66/66 [==============================] - 0s 544us/step - loss: 0.4210 - acc: 0.8939 - val_loss: 0.5247 - val_acc: 0.7500
Epoch 62/100
66/66 [==============================] - 0s 786us/step - loss: